In [3]:
import cv2
import numpy as np
import os 
import glob

In [18]:
def to_homogeneous(points_2d):

    return np.hstack([points_2d, np.ones((points_2d.shape[0],1))])


def compute_normalization_transform_2D(homogeneous_2D):

    centroid, scale = np.mean(homogeneous_2D[:,:2], axis = 0), np.std(homogeneous_2D[:,:2])
    scale = np.sqrt(2) / scale

    transformation = np.array([[scale,0, -(scale*centroid[0])],[0, scale, -(scale*centroid[1])], [0,0,1]])
    return transformation

def apply_K_inv(K_inv, homogenous_points):
    return np.dot(K_inv, homogenous_points.T).T

def normalize_points(transform, homogenous_points):
    return np.dot(transform, homogenous_points.T).T

def estimate_fundamental_matrix(x_prime, x):

    x_prime_homogeneous = to_homogeneous(x_prime)
    x_homogeneous       = to_homogeneous(x)

    
    transform_x_prime = compute_normalization_transform_2D(x_prime_homogeneous)
    transform_x       = compute_normalization_transform_2D(x_homogeneous)

    x_prime_camera_norm = normalize_points(transform_x_prime, x_prime_homogeneous)
    x_camera_norm = normalize_points(transform_x, x_homogeneous)
    N_points = x_prime.shape[0]
    A = np.zeros((N_points, 9))

    for p in range(N_points):
        A[p,:] = np.kron(x_prime_camera_norm[p], x_camera_norm[p])
    
    U,S,Vh = np.linalg.svd(A)
    F_norm = Vh[-1].reshape(3,3)
    Uf, Sf, Vf = np.linalg.svd(F_norm)
    Sf[-1] = 0
    F_norm_corrected = Uf @ np.diag(Sf) @ Vf
    F_norm_corrected = F_norm_corrected / F_norm_corrected[2,2]

    F = np.matmul(transform_x_prime.T, np.matmul(F_norm_corrected, transform_x))

    return F / F[2,2]    

def estimateEssentialFromFundamental(fundamental, k):

    E = np.matmul(k.T, np.matmul(fundamental,k))
    U,S,V = np.linalg.svd(E)
    E_estimated = U @ np.diag([1,1,0]) @ V

    return E_estimated / E_estimated[2,2]

def estimate_essential_matrix(x_prime, x, K):

    x_prime_homogeneous = to_homogeneous(x_prime)
    x_homogeneous       = to_homogeneous(x)

    K_inv = np.linalg.inv(K)
    x_prime_camera = apply_K_inv(K_inv, x_prime_homogeneous)
    x_camera       = apply_K_inv(K_inv, x_homogeneous)

    transform_x_prime = compute_normalization_transform_2D(x_prime_camera)
    transform_x       = compute_normalization_transform_2D(x_camera)

    x_prime_camera_norm = normalize_points(transform_x_prime, x_prime_camera)
    x_camera_norm = normalize_points(transform_x, x_camera)
    
    N_points = x_prime.shape[0]
    A = np.zeros((N_points, 9))

    for p in range(N_points):
        A[p,:] = np.kron(x_prime_camera_norm[p], x_camera_norm[p])
    
    U,S,Vh = np.linalg.svd(A)
    E_norm = Vh[-1].reshape(3,3)

    Ue, Se, Ve = np.linalg.svd(E_norm)
    Se = [1, 1, 0]

    E_norm_corrected = Ue @ np.diag(Se) @ Ve
    E_norm_corrected = E_norm_corrected / E_norm_corrected[2,2]

    E = np.matmul(transform_x_prime.T, np.matmul(E_norm_corrected, transform_x))

    return E / E[2,2]    


def decomposeEssentialMat(E):

    U,S,V = np.linalg.svd(E)

    W = np.array([[0, -1, 0], [1, 0, 0], [0, 0, 1]])

    t1, t2 = U[:,2], -U[:,2]
    R1 = np.matmul(U, np.matmul(W,V))
    R2 = np.matmul(U, np.matmul(W.T, V))

    # Four possible decompositions
    rotations = []
    translations = []

    if np.linalg.det(R1) < 0:

        rotations.append(-R1)
        rotations.append(-R1)

        translations.append(-t1)
        translations.append(-t2)
    else:
        rotations.append(R1)
        rotations.append(R1)

        translations.append(t1)
        translations.append(t2)

    if np.linalg.det(R2) < 0:

        rotations.append(-R2)
        rotations.append(-R2)

        translations.append(-t1)
        translations.append(-t2)
    else:
        rotations.append(R2)
        rotations.append(R2)

        translations.append(t1)
        translations.append(t2)

    return rotations, translations

def computeProjectionMatrix(K,R,t):

    extrinsic = np.zeros((3,4))
    extrinsic[:,:3] = R
    extrinsic[:,3]  = t

    print(extrinsic)

    return np.matmul(K, extrinsic)



In [5]:
correspondence_file = "/home/parikshat/MyWorkings/Masters/Semester_2/3d_vision/Project/Stage_1_Data_ver._2/stage1/boot/correspondences/0_2.txt"
correspondences = np.loadtxt(correspondence_file)

In [6]:
import json 
with open('/home/parikshat/MyWorkings/Masters/Semester_2/3d_vision/Project/Stage_1_Data_ver._2/stage1/boot/camera_parameters.json') as json_data:
    camera_parameters = json.load(json_data)
    json_data.close()
    

In [7]:
K     = np.array(camera_parameters['intrinsics'])
K_inv = np.linalg.inv(K) 

In [8]:
pts1 = correspondences[:,:2]
pts2 = correspondences[:,2:]

In [12]:
random_8_points = np.random.choice(pts1.shape[0], 8)

x_prime = pts1[random_8_points]
x       = pts2[random_8_points]

In [221]:
fundamental_estimated = estimate_fundamental_matrix(x_prime,x, )

In [222]:
fundamental_estimated

array([[-2.04351419e-06, -2.72221285e-05,  3.22568114e-03],
       [ 3.19569503e-05,  7.53778010e-06, -1.36955431e-02],
       [-3.03961599e-03,  2.72712507e-03,  1.00000000e+00]])

In [223]:
E = np.matmul(K.T, np.matmul(fundamental_estimated, K))

In [224]:
E / E[2,2]

array([[  0.89276225,  11.89269382,   6.31914537],
       [-13.96122369,  -3.293075  ,  -9.90174938],
       [ -5.76385699,   9.09383939,   1.        ]])

In [9]:
x_prime_homo = to_homogeneous(pts1)
x_homo       = to_homogeneous(pts2)
x_p_cam = apply_K_inv(K_inv, x_prime_homo)
x_cam   = apply_K_inv(K_inv, x_homo)
N_points = pts1.shape[0]

In [10]:
best = None
best_criterion = 0
for i in range(1000):

    points = np.random.choice(N_points,8, replace= False)
    
    f_matrix = estimateEssentialFromFundamental(estimate_fundamental_matrix(pts1[points], pts2[points]),K)
    criterion = np.sum(np.abs((x_p_cam.T * (f_matrix @ x_cam.T)).sum(axis = 0)) < 0.001)

    if criterion > best_criterion:
        best_criterion = criterion
        best = f_matrix

In [20]:
rotations, translations = decomposeEssentialMat(f_matrix)

In [21]:
translations

[array([0.16269119, 0.16157188, 0.97335816]),
 array([-0.16269119, -0.16157188, -0.97335816]),
 array([0.16269119, 0.16157188, 0.97335816]),
 array([-0.16269119, -0.16157188, -0.97335816])]